In [1]:
import numpy as np
import scipy.integrate as nint
import scipy.interpolate as interp
import scipy.stats as stats
import scipy.optimize as opt
import matplotlib.pyplot as plt
from tqdm import tqdm
import integrator as intt
import analysis as an
import likelihood as lh
import emcee as mc

%matplotlib inline

In [2]:
star_Cat = "G"
starFile = '/Users/john/Desktop/john_code/DarkDisk/'+star_Cat+'_stars.txt'
zfile_columnZ = 5
zfile_columnEVFS = 6
zfile_columnZErr = 16

sunZ = -0.326
rhoDHalo = 0.015
sunW = 6.395
h_DD = 10.; sig_DD = 0.5;

sigmaH2 = 3.7; rhoH2 = 0.0104;
sigmaHI1 = 7.1; rhoHI1 = 0.0277;
sigmaHI2 = 22.1; rhoHI2 = 0.0073;
sigmaWGas = 39.0; rhoWGas = 0.0005;
sigmaGiants = 15.5; rhoGiants = 0.0006;
sigmaMV2p5 = 7.5; rhoMV2p5 = 0.0018;
sigma3MV4 = 12.0; rho3MV4 = 0.0018;
sigma4MV5 = 18.0; rho4MV5 = 0.0029;
sigma5MV8 = 18.5; rho5MV8 = 0.0072;
sigmaMV8 = 18.5; rhoMV8 = 0.0216;
sigmaWDwarfs = 20.0; rhoWDwarfs = 0.0056;
sigmaBDwarfs = 20.0; rhoBDwarfs = 0.0015;
sigmaDHalo = float('nan')

err_rhoDHalo = 0.005; err_rho_star_sig = 0.28
err_sigmaH2 = 0.2; err_rhoH2 = 0.00312;
err_sigmaHI1 = 0.5; err_rhoHI1 = 0.00554;
err_sigmaHI2 = 2.4; err_rhoHI2 = 0.0007;
err_sigmaWGas = 4.0; err_rhoWGas = 0.00003;
err_sigmaGiants = 1.6; err_rhoGiants = 0.00006;
err_sigmaMV2p5 = 2.0; err_rhoMV2p5 = 0.00018;
err_sigma3MV4 = 2.4; err_rho3MV4 = 0.00018;
err_sigma4MV5 = 1.8; err_rho4MV5 = 0.00029;
err_sigma5MV8 = 1.9; err_rho5MV8 = 0.00072;
err_sigmaMV8 = 4.0; err_rhoMV8 = 0.0028;
err_sigmaWDwarfs = 5.0; err_rhoWDwarfs = 0.001;
err_sigmaBDwarfs = 5.0; err_rhoBDwarfs = 0.0005;

sigma_err = np.array([err_sigmaH2, err_sigmaHI1, err_sigmaHI2, err_sigmaWGas, err_sigmaGiants, err_sigmaMV2p5, err_sigma3MV4, err_sigma4MV5, err_sigma5MV8, err_sigmaMV8, err_sigmaWDwarfs, err_sigmaBDwarfs])
rho_err = np.array([err_rhoH2, err_rhoHI1, err_rhoHI2, err_rhoWGas, err_rhoGiants, err_rhoMV2p5, err_rho3MV4, err_rho4MV5, err_rho5MV8, err_rhoMV8, err_rhoWDwarfs, err_rhoBDwarfs])

sigma = np.array([sigmaH2, sigmaHI1, sigmaHI2, sigmaWGas, sigmaGiants, sigmaMV2p5, sigma3MV4, sigma4MV5, sigma5MV8, sigmaMV8, sigmaWDwarfs, sigmaBDwarfs])
rho = np.array([rhoH2, rhoHI1, rhoHI2, rhoWGas, rhoGiants, rhoMV2p5, rho3MV4, rho4MV5, rho5MV8, rhoMV8, rhoWDwarfs, rhoBDwarfs])

mean_param = np.array([ 14.6, rhoDHalo])
mean_param = np.append(mean_param, sigma)
mean_param = np.append(mean_param, rho)

err_param = np.array([err_rho_star_sig, err_rhoDHalo])
err_param = np.append(err_param, sigma_err)
err_param = np.append(err_param, rho_err)

In [3]:
wdata = an.fetchWData(starFile, wSun = sunW, show_plot = False, verbose = False)
star_sig, star_sig_err = lh.fetchWDist_fast(wdata, verbose=False)
zdata = np.loadtxt(starFile, delimiter= ",", skiprows=1, usecols=(zfile_columnZ, zfile_columnZErr, zfile_columnEVFS), unpack=True)  

param = star_sig[0], sunZ, rhoDHalo, sigmaH2, sigmaHI1, sigmaHI2, sigmaWGas, sigmaGiants,\
sigmaMV2p5, sigma3MV4, sigma4MV5, sigma5MV8, sigmaMV8, sigmaWDwarfs, sigmaBDwarfs, rhoH2, rhoHI1, rhoHI2,\
rhoWGas, rhoGiants, rhoMV2p5, rho3MV4, rho4MV5, rho5MV8, rhoMV8, rhoWDwarfs, rhoBDwarfs

%time loglikelihood = lh.loglikelihood(param, h_DD, sig_DD, zdata, mean_param=mean_param, err_param=err_param)
print(loglikelihood)

CPU times: user 766 ms, sys: 16.7 ms, total: 783 ms
Wall time: 808 ms
inf


In [4]:
epsilon = 0.0001
#initial_guess = np.array([h_DD, sig_DD, 14.6])
#fixed_params = np.append(sigma, rho)
#fixed_params = np.append([sunZ, rhoDHalo], fixed_params)

initial_guess = np.array([14.6, sunZ, rhoDHalo])
initial_guess = np.append(initial_guess, sigma)
initial_guess = np.append(initial_guess, rho)

print("parameter space dimension: " + str(len(initial_guess)))
bounds = [(14.3, 14.9), (None, None), (epsilon, None),\
         (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None),\
         (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None),\
         (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None),\
         (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None), (epsilon, None)]

#%time result = opt.minimize(lh.loglikelihood_test, initial_guess, args=(fixed_params, zdata), bounds = bounds)
%time result = opt.minimize(lh.loglikelihood, initial_guess, args=(h_DD, sig_DD, zdata, mean_param, err_param), bounds = bounds)
x_return = result["x"]

print("return parameters: " + str(x_return) )
print("minimized likelihood: " + str(lh.loglikelihood(x_return, h_DD, sig_DD, zdata, mean_param, err_param)) )

parameter space dimension: 27


likelihood.py:79: RuntimeWarning: invalid value encountered in less
  z_relevant_indx = ( np.abs(z_coord-z) < 3.*zerr )
likelihood.py:208: RuntimeWarning: invalid value encountered in divide
  starDensity = starDensity/starDensity_norm
likelihood.py:209: RuntimeWarning: divide by zero encountered in divide
  starDensity_Delta = SDDensity/starDensity_norm


ValueError: cannot convert float NaN to integer

In [ ]:
interpz = []
interpy = []
interpx = []
interpxy = []

HD_step = 40
SgmD_step = 2
for nHD in tqdm(range(1,400,HD_step)):
   for nSgmD in range(1,35,SgmD_step):
      h_DD = (nHD-0.8)
      sig_DD = (nSgmD-0.5)
      result = opt.minimize(lh.loglikelihood, initial_guess, args=(h_DD, sig_DD, zdata, mean_param, err_param), bounds = bounds)
      hdd_ret, Sigdd_ret, llh_rett = lh.diskParamLlhReturn(x_return, h_DD, sig_DD, zdata, mean_param, err_param)
      interpx.append( Sigdd_ret )
      interpy.append( hdd_ret )
      interpxy.append( [Sigdd_ret, hdd_ret] )
      interpz.append( llh_rett )

In [ ]:
border_x = 100
border_y = 20

plt.scatter(interpy, interpx)
plt.scatter(np.array([border_x for k in range(border_y)]),np.array([k for k in range(border_y)]),color='black')
plt.scatter(np.array([k for k in range(border_x)]),np.array([border_y for k in range(border_x)]),color='black')
plt.title('data point coverage'); plt.ylabel('$\Sigma_D$'); plt.xlabel('$h_D$');
plt.show()

spacex, spacey = np.mgrid[0:20:42j,0:100:25j]

f_likely = interp.griddata(np.array(interpxy), np.array(interpz), (spacex,spacey), method='nearest')

In [ ]:
cl_contour = [-2.71]


outputx = spacex[:,0]
outputy = spacey[0,:]
outputz = np.array([])
opt_y_coord = np.array([])

for j, y in enumerate(outputy):
   peak_f = np.amax( f_likely[:,j] )
   opt_y_coord = np.append(opt_y_coord, peak_f)

plot_z = np.empty_like(f_likely)

for i, x in enumerate(outputx):
   for j, y in enumerate(outputy):
      plot_z[i,j] = 2.*(f_likely[i,j] - opt_y_coord[j])

spacex = np.transpose(spacex)
spacey = np.transpose(spacey)
plot_z = np.transpose(plot_z)
plt.pcolor(spacey, spacex, plot_z, vmin=plot_z.min(), vmax=plot_z.max())
plt.colorbar()
plt.contour(spacey, spacex, plot_z, cl_contour)
plt.xlabel('$h_D$'); plt.ylabel('$\Sigma_D$'); plt.title('Exclusion Contour')
plt.show()